<a href="https://colab.research.google.com/github/krdeepak39/Deep-learning-model/blob/main/GANS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU  # mean some value be negative
from keras.layers import Dropout
from matplotlib import pyplot
import matplotlib.pyplot as plt

In [ ]:

def define_discriminator(in_shape=(28,28,1)):
  model = Sequential()
  model.add(Conv2D(32,(3,3), strides  = (2,2), padding='same', input_shape = in_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.4))
  model.add((Conv2D(64,(3,3), strides  =(2,2), padding = 'same')))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.4))
  model.add(Flatten())
  model.add(Dense(1,activation = 'sigmoid'))

  opt = Adam (lr = 0.0002, beta_1=0.5)
  model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

  return model


In [ ]:
def define_genrator (latent_dim):
  gen_model = Sequential()
  n_nodes = 128*7*7
  gen_model.add(Dense(n_nodes, input_dim = latent_dim))
  gen_model.add(LeakyReLU(alpha=0.2))
  gen_model.add(Reshape((7,7,128)))
  #upsample to 14X14
  gen_model.add(Conv2DTranspose(128,(4,4), strides = (2,2), padding='same'))
  gen_model.add(LeakyReLU(alpha=0.2))
  #upsample to 28X28
  gen_model.add(Conv2DTranspose(128,(4,4), strides = (2,2), padding='same'))
  gen_model.add(LeakyReLU(alpha=0.2))
  gen_model.add(Conv2D(1,(7,7), activation = 'sigmoid', padding ='same'))

  return gen_model

In [ ]:
def define_gan(g_model,d_model):
  d_model.trainable= False
  model = Sequential()
  model.add(g_model)
  model.add(d_model)

  opt = Adam(lr =0.0002, beta_1 = 0.5)
  model.compile (loss = 'binary_crossentropy', optimizer=opt)

  return model


In [ ]:
def load_real_samples():
    (trainX, _), (_, _) = load_data()
    X = expand_dims(trainX, axis=-1)
    X = X.astype('float32')
    X = X/255.0
    return X

In [ ]:
def generate_real_samples (dataset, n_samples):
  ix = randint(0, dataset.shape[0], n_samples)
  X =dataset[ix]
  y= ones((n_samples,1))
  return X,y

In [ ]:
def generate_latent_points(latent_dim, n_samples):
  X_input = randn( latent_dim*n_samples)
  X_input =X_input.reshape(n_samples, latent_dim)
  return X_input

In [ ]:
def generate_fake_samples(g_model, latent_dim, n_samples):
  X_input = generate_latent_points(latent_dim, n_samples)
  X = g_model.predict(X_input)
  y = zeros((n_samples,1))
  return X,y

In [ ]:
def viz_plot(examples,epoch, n=10):
  for i in range(n*n):
    pyplot.subplot (n,n,1+i)
    pyplot.axis('off')
    pyplot.imshow(examples[i, :, :, 0], cmap = 'gray_r')
    filename = 'drive/MyDrive/Colab Notebooks/generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()

    return pyplot

In [ ]:
def summary(g_model, d_model, dataset, latent_dim, epoch, n_samples = 100):
    X_real, y_real = generate_real_samples(dataset, n_samples)

    _, acc_real = d_model.evaluate(X_real, y_real, verbose = 0)

    X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)

    _, acc_fake = d_model.evaluate(X_fake, y_fake, verbose = 0)

    print('> Accuracy real: %.0f%%, fake: %0.0f%%' % (acc_real*100, acc_fake*100))

    pyplot = viz_plot(X_fake, epoch)

    return pyplot

In [ ]:
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs = 25, n_batch = 256):
    bat_per_epoch = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch/2)
    pyplot = []

    for i in range(n_epochs):
        for j in range(bat_per_epoch):
            X_real, y_real = generate_real_samples(dataset, half_batch)

            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)

            d_loss_real, _ = d_model.train_on_batch(X_real, y_real)
            d_loss_fake, _ = d_model.train_on_batch(X_fake, y_fake)

            d_loss = d_loss_real + d_loss_fake

            X_gan = generate_latent_points(latent_dim, n_batch)

            y_gan = ones((n_batch, 1))

            g_loss = gan_model.train_on_batch(X_gan, y_gan)

            print('> %d, %d/%d, d = %.3f, g=%.3f' % (i+1, j+1, bat_per_epoch, d_loss, g_loss))

        if (i) % 10 == 0:
            pyplot.append(summary(g_model, d_model, dataset, latent_dim, i))

    return pyplot

In [ ]:
latent_dim = 100

d_model = define_discriminator()

g_model = define_genrator(latent_dim)

gan_model = define_gan(g_model, d_model)

dataset = load_real_samples()

plot = train(g_model, d_model, gan_model, dataset, latent_dim)

Streaming output truncated to the last 5000 lines.
4/4 [==============================] - 1s 144ms/step
> 3, 51/234, d = 1.405, g=0.698
4/4 [==============================] - 1s 227ms/step
> 3, 52/234, d = 1.397, g=0.701
4/4 [==============================] - 1s 141ms/step
> 3, 53/234, d = 1.410, g=0.702
4/4 [==============================] - 1s 231ms/step
> 3, 54/234, d = 1.397, g=0.703
4/4 [==============================] - 1s 141ms/step
> 3, 55/234, d = 1.397, g=0.705
4/4 [==============================] - 1s 230ms/step
> 3, 56/234, d = 1.389, g=0.709
4/4 [==============================] - 1s 141ms/step
> 3, 57/234, d = 1.389, g=0.709
4/4 [==============================] - 1s 224ms/step
> 3, 58/234, d = 1.397, g=0.702
4/4 [==============================] - 1s 140ms/step
> 3, 59/234, d = 1.406, g=0.709
4/4 [==============================] - 1s 228ms/step
> 3, 60/234, d = 1.401, g=0.705
4/4 [==============================] - 1s 160ms/step
> 3, 61/234, d = 1.410, g=0.702
4/4 [=========